## 範例資料集

Sea turtle sightings in Taiwan | 台灣海龜目擊紀錄

https://n2t.net/ark:37281/k5f374z25

In [ ]:
import json

import pandas as pd
from pandas import json_normalize
import plotly.express as px


pd.options.plotting.backend = "plotly"

# Basic Search API
ENDPOINT = "https://data.depositar.io/api/3/action/datastore_search?resource_id=661e0cc0-3567-4fa3-9d6e-1fe56f53d05d&limit=10000"
# SQL Search API
ENDPOINT_SQL = "https://data.depositar.io/api/3/action/datastore_search_sql"

# Example SQL query: find sightings in Taitung County
sql = "SELECT * FROM \"661e0cc0-3567-4fa3-9d6e-1fe56f53d05d\" WHERE county = 'Taitung'"

## 取得資料



In [ ]:
from urllib.request import urlopen
import requests


# Get all data via Basic Search API
data = urlopen(ENDPOINT)
dic = json.loads(data.read())

# Get sightings in Taitung County via SQL Search API
data2 = requests.get(ENDPOINT_SQL, params={"sql": sql})
dic2 = data2.json()

## 瀏覽所有資料

In [ ]:
# Parse JSON
df = json_normalize(dic["result"]["records"])
df

## 縣市別目擊數

In [ ]:
# Group by county
county_df = df.groupby(["county"]).count()

# Show total individual counts for each county
county_df["y"] = 0
fig1 = px.bar(county_df, x="individualCount", y="y", text = "individualCount",
              color=county_df["individualCount"].keys(), orientation="h")
fig1.update_yaxes(showticklabels=False, title=None)
fig1.update_xaxes(range=[0, 3500])
fig1.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig1.show()

## 台東縣目擊地圖（使用進階 SQL 搜尋 API 取得台東縣資料）

In [ ]:
# Parse JSON
df2 = json_normalize(dic2["result"]["records"])

# Convert string to float
df2["decimalLongitude"] = df2["decimalLongitude"].astype(float)
df2["decimalLatitude"] = df2["decimalLatitude"].astype(float)

# Show sightings in Taitung County, with occurrenceID, eventDate and associatedReferences
fig2 = px.scatter_mapbox(df2, lat="decimalLatitude", lon="decimalLongitude",
                         hover_name="occurrenceID",
                         hover_data=["eventDate", "associatedReferences"],
                         mapbox_style="open-street-map")
fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig2.show()